In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

#year = 2022
name = 'BANPU'
buy_date = '2018-06-21'

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND buys.date >= '%s'
ORDER BY sell_year DESC, sell_month DESC, name'''

sql = sql % (name, buy_date)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'BANPU' AND buys.date >= '2018-06-21'
ORDER BY sell_year DESC, sell_month DESC, name


In [4]:
sells_df

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit,kind
0,BANPU,2020,8,2020,6,5.45,6.35,-0.90,6000,"32,700.00","38,100.00","-5,400.00",-14.17,"-5,556.82",NBS
1,BANPU,2020,8,2018,9,5.45,18.10,-12.65,12000,"65,400.00","217,200.00","-151,800.00",-69.89,"-152,425.93",NBS
2,BANPU,2020,5,2018,6,6.50,18.10,-11.60,6000,"39,000.00","108,600.00","-69,600.00",-64.09,"-69,926.92",NBS
3,BANPU,2020,5,2018,6,7.40,18.10,-10.70,6000,"44,400.00","108,600.00","-64,200.00",-59.12,"-64,538.88",NBS


### Record selection for active stocks

In [5]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
ORDER BY buys.date DESC'''
sql = sql % name
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,unit_cost,qty,cost_amt,net
0,BANPU,2022-09-08,14.00,"7,500","105,000.00","105,232.56"


In [18]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
AND buys.date >= '2018-06-21'
ORDER BY buys.date DESC'''
sql = sql % name
print(sql)

buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)


SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'BANPU' AND status = 'Active'
AND buys.date >= '2018-06-21'
ORDER BY buys.date DESC


,name,buy_date,unit_cost,qty,cost_amt,net
0,BANPU,2022-09-08,14.00,"7,500","105,000.00","105,232.56"


In [19]:
buys_df[['cost_amt','net','qty']].sum()

cost_amt   105,000.00
net        105,232.56
qty          7,500.00
dtype: float64

In [20]:
sql = '''
SELECT name, p_date, amt, net, amt-net AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = '%s' AND x_date > '%s'
ORDER BY p_date DESC'''
sql = sql % (name, buy_date)
print(sql)

div_df = pd.read_sql(sql, conpf)
div_df


SELECT name, p_date, amt, net, amt-net AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = 'BANPU' AND x_date > '2018-06-21'
ORDER BY p_date DESC


,name,p_date,amt,net,refund,year,q,x_date,number,ppu,div_1k
0,BANPU,2019-09-25,"3,600.00","3,240.00",360.00,2019,2,2019-09-10,24000,0.15,135.00
1,BANPU,2019-04-30,"7,350.00","6,615.00",735.00,2018,4,2019-04-30,21000,0.35,315.00
2,BANPU,2018-09-28,"4,200.00","3,780.00",420.00,2018,2,2018-09-28,12000,0.35,315.00


In [21]:
div_grp = div_df.groupby(['name'], as_index=True).agg(
    {
        'amt':['sum'],
        'net':['sum'], 
        'number':['mean','count'],         
    }
)
div_grp

amt       net    number      
            sum       sum      mean count
name                                     
BANPU 15,150.00 13,635.00 19,000.00     3

In [22]:
acc_div = 13_635
acc_div

13635

In [23]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
AND buys.date >= '2018-06-21'
ORDER BY name, buys.date'''
sql = sql % name
print(sql)

buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)


SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'BANPU' AND status = 'Active'
AND buys.date >= '2018-06-21'
ORDER BY name, buys.date


,name,buy_date,unit_cost,qty,cost_amt,net
0,BANPU,2022-09-08,14.00,"7,500","105,000.00","105,232.56"


In [24]:
buys_df[['qty','cost_amt']].sum()

qty          7,500.00
cost_amt   105,000.00
dtype: float64

In [13]:
qty = 15_000
cost_amt = 465_000
acc_div = 25_500
sell_amt = cost_amt - acc_div
sell_amt

439500

In [14]:
sell_price = sell_amt / qty
sell_price

29.3

In [15]:
#unit_plus = 2.75
sell_price = 29.50 # + unit_plus
sales_amt = sell_price * qty
sales_amt

442500.0

### Plan A: Sell at 29.50 x 15,000 shares